# Importing packages

In [1]:
using JuMP
using GLPK
using Formatting
using GLPKMathProgInterface

# Reading problem instance and defining Constants

In [2]:
instance_file = "Problema2/smt_7_6.dat"
time_limit = 60 * 30; # seconds

In [4]:
f = open(instance_file, "r")         
 
lines = readlines(f)
 
close(f)

n, m, s, t = [parse(Int, num_char) for num_char in split(lines[1])]
print("number of vertices (n): $n\n")
print("number of edges (m): $m\n")
print("initial vertice (s): $s\n")
print("final vertice (t): $t\n")

V = Array(1:n)
V_st = copy(V)
deleteat!(V_st, V_st .== s)
deleteat!(V_st, V_st .== t)
print("\nvertices (V): $V\n")
print("intermidiate vertices (V_st): $V_st\n")

D = zeros(Int64, (n, n))
G = zeros(Int64, (n, n))
P = zeros(Int64, (n, n, n))

print("\nEdges and weights: \n")
for i in 2:m+1
    u, v, d = [parse(Int, num_char) for num_char in split(lines[i])]
    print("{$u, $v} = $d\n")
    D[u, v] = d
    D[v, u] = d
    G[u, v] = 1
    G[v, u] = 1
end

for i in 1:n
    for j in 1:n
        for k in 1:n
            P[i,j,k] = abs(D[i,j] - D[j,k])
        end
    end
end

M = maximum(P);

number of vertices (n): 7
number of edges (m): 6
initial vertice (s): 1
final vertice (t): 6

vertices (V): [1, 2, 3, 4, 5, 6, 7]
intermidiate vertices (V_st): [2, 3, 4, 5, 7]

Edges and weights: 
{7, 2} = 75
{4, 1} = 13
{4, 3} = 48
{6, 2} = 10
{5, 1} = 93
{7, 4} = 59


# Defining Model

In [9]:
model = Model(GLPK.Optimizer)

set_time_limit_sec(model, time_limit);

@variable(model, C[1:n, 1:n], Bin)
@variable(model, Q[1:n, 1:n, 1:n], Bin)
@variable(model, p >= 0, Int)

@objective(model, Min, p)

# Restrições para gerar um caminho C subgrafo de G
@constraint(model, sum(C[j,s] for j = 1:n) == 0)
@constraint(model, sum(C[s,j] for j = 1:n) == 1)
@constraint(model, sum(C[j,t] for j = 1:n) == 1)
@constraint(model, sum(C[t,j] for j = 1:n) == 0)
@constraint(model, [i in V_st], sum(C[j,i] for j = 1:n) <= 1)
@constraint(model, [i in V_st], sum(C[j,i] for j = 1:n) - sum(C[i,j] for j = 1:n) == 0)
@constraint(model, [i = 1:n, j=1:n], C[i,j] - G[i,j] <= 0)

# Restrições para que  Q_{ijk} = C_{ij} ^ C_{jk}
@constraint(model, [i = 1:n, j=1:n, k=1:n], Q[i,j,k] <= (C[i,j] + C[j,k])/2)
@constraint(model, [i = 1:n, j=1:n, k=1:n], Q[i,j,k] >= C[i,j] + C[j,k] - 1)

# Restrições para que p seja o passo máximo do caminho selecionado
@constraint(model, [i = 1:n, j=1:n, k=1:n], p >= P[i,j,k] + (-M * (1 - Q[i,j,k])));

# Optimizing

In [10]:
optimize!(model)
print(objective_value(model))

65.0